In [0]:
# importing library or modules# 
import os
import json
import boto3


In [0]:
# Specify the DBFS path to the file
file_path = "/Workspace/Cred/secrets.ipynb"

# Check if the file exists
if not os.path.exists(file_path):
    print(f"File {file_path} does not exist.")
else:
    # # Read the content of the file
    # with open(file_path, 'r') as file:
    #     file_content = file.read()

    # # Display the content of the file
    # print(file_content)
    # Read the content of the file
    with open(file_path, 'r') as file:
        notebook_content = file.read()

    # Parse the JSON content
    notebook_json = json.loads(notebook_content)
    # aws_access_key = notebook_json['aws_access_key']
    # aws_secret_access_key = notebook_json['aws_secret_access_key']
        
    # Extract the source lines from the first cell
    source_lines = notebook_json['cells'][0]['source']

    # Combine lines into a single string
    code_str = ''.join(source_lines)

    # Now you can parse the string to extract the keys using regex
    import re

    access_key_match = re.search(r'aws_access_key\s*=\s*"([^"]+)"', code_str)
    secret_key_match = re.search(r'aws_secret_key\s*=\s*"([^"]+)"', code_str)

    aws_access_key = access_key_match.group(1) if access_key_match else None
    aws_secret_key = secret_key_match.group(1) if secret_key_match else None



In [0]:
# create_session
session = boto3.Session(
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name='us-west-2'
)

In [0]:
s3 = session.client('s3')

In [0]:

bucket_name = 'ind-wiki-scrape-1'  # valid bucket name (lowercase, hyphens)



# For us-east-1, no CreateBucketConfiguration is needed
if session.region_name == 'us-east-1':
    # Check if the bucket exists
    existing_buckets = s3.list_buckets()
    if not any(bucket['Name'] == bucket_name for bucket in existing_buckets['Buckets']):
        s3.create_bucket(Bucket=bucket_name)
else:
    # Check if the bucket exists
    existing_buckets = s3.list_buckets()
    if not any(bucket['Name'] == bucket_name for bucket in existing_buckets['Buckets']):
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': session.region_name}
        )


response = s3.list_buckets()
for bucket in response['Buckets']:
    print(bucket['Name'])